## Predicting Future Sales

You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

### Data Description
#### File descriptions
* sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
* test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
* sample_submission.csv - a sample submission file in the correct format.
* items.csv - supplemental information about the items/products.
* item_categories.csv - supplemental information about the items categories.
* shops.csv- supplemental information about the shops.

#### Data fields
* ID - an Id that represents a (Shop, Item) tuple within the test set
* shop_id - unique identifier of a shop
* item_id - unique identifier of a product
* item_category_id - unique identifier of item category
* item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
* item_price - current price of an item
* date - date in format dd/mm/yyyy
* date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* item_name - name of item
* shop_name - name of shop
* item_category_name - name of item category

In [ ]:
# Import all the important    
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import optimize, stats    
%matplotlib inline

### Read all the data files

In [ ]:
df_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
df_shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
df_items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
df_item_categories = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
df_test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
df_train.head(4)

In [ ]:
df_shops.head(4)


In [ ]:
df_items.head(4)

In [ ]:
df_item_categories.head(4)

In [ ]:
df_test.head(4)

In [ ]:
# we check data type of all columns
df_train.info()

In [ ]:
#date is object so we will change Dtype from object to datetime64
df_train['date']=pd.to_datetime(df_train['date'])

In [ ]:
#now we check if there is any null values present in dataset
df_train.isnull().sum()

In this data set date format is 02-10-2015 but we only need month and year
SO we delete date day and keep only month and year

In [ ]:
df_train['date']=df_train['date'].dt.strftime('%Y-%m')

In [ ]:
df_train.head().sort_values(by='date')

Now,in dataset 'date_block_num' = consicutive month number and 'item_price' is given
But for our prediction we do not need these columns so we will deletd these columns

In [ ]:
df_train.drop(['date_block_num','item_price'] , axis =1, inplace= True)

In [ ]:
df_train.head().sort_values(by='date')

Now we will groupby our data with reference to date, shop_id, item_id

In [ ]:
df=df_train.groupby(['date','shop_id','item_id']).sum()
df.head()

Now after group by we will make pivot table for easy understanding 
we keep shop_id and item_id as index
we keep date as columns and fill item_cnt_day in value

In [ ]:
df = df.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
df.reset_index(inplace=True)
df.head()

In [ ]:
df_test= pd.merge(df_test , df , on = ['shop_id', 'item_id'], how = 'left')
df_test.drop(['ID', '2013-01'], axis =1, inplace=True)
df_test= df_test.fillna(0)

In [ ]:
df_test.head()

## Machine Learning Model

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample
from sklearn.pipeline import Pipeline

### Training and Testing Data

In [ ]:
Y_train = df['2015-10'].values
X_train = df.drop(['2015-10'], axis = 1)
X_test = df_test

In [ ]:
x_train, x_test, y_train, y_test = train_test_split( X_train, Y_train, test_size=0.2, random_state=101)

### Linear Regression

In [ ]:
LR = LinearRegression()
LR.fit(x_train,y_train)

print('Train set mse:', mean_squared_error(y_train, LR.predict(x_train)))
print('Test set mse:', mean_squared_error(y_test, LR.predict(x_test)))
print('Test set score:', LR.score(x_train,y_train))

In [ ]:
RFR = RandomForestRegressor(n_estimators = 100)
RFR.fit(x_train,y_train)

print('Train set mse:', mean_squared_error(y_train, RFR.predict(x_train)))
print('Test set mse:', mean_squared_error(y_test, RFR.predict(x_test)))
print('Test set score:', RFR.score(x_train,y_train))

In [ ]:
prediction = RFR.predict(X_test)

In [ ]:
prediction = list(map(round, prediction))

In [ ]:
df_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
print(df_submission.shape)
df_submission.head()

In [ ]:
df_submission['item_cnt_month'] = prediction
df_submission.to_csv('prediction.csv', index=False)
df_submission.head()